<div class="alert alert-block alert-info">
                                            Regular Expressions
    <p><a href= "https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html">Pandas documentation for text</a>
    <p><a href = 'https://www.analyticsvidhya.com/blog/2014/11/text-data-cleaning-steps-python/'> Analytics Vidhya Data Cleaning</a> 
    <p><a href = 'http://ieva.rocks/2016/08/07/cleaning-text-for-nlp/'> NLP Data Cleaning</a> 
        
</div>

In [ ]:
from IPython.display import Audio, Image, Video

In [ ]:
Image('FIG1.PNG', width=500, height=500)

In [ ]:
Image('FIG2.PNG', width=500, height=500)


<div class='alert alert-block alert-info'>
<li>[^] ( within [ ] ) is negation in regular expression whereas ^ is "begining of string"</li>

<li>[^a-z] matches any single character that is not from "a" to "z"</li>

<li>^[a-z] means string starts with from "a" to "z"</li>

</div>

In [ ]:
Video("RegEx.mp4", width=400, height=400)

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.iloc[0]

'03/25/93 Total time of visit (in minutes):\n'

In [2]:
pd.set_option('max_colwidth', 200)
pd.set_option('max_rows', 700)

In [3]:
df.shape
df.iloc[0:7]

0                        03/25/93 Total time of visit (in minutes):\n
1                                      6/18/85 Primary Care Doctor:\n
2            sshe plans to move as of 7/8/71 In-Home Services: None\n
3                               7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
5                                   .Per 7/06/79 Movement D/O note:\n
6      4, 5/18/78 Patient's thoughts about current substance abuse:\n
dtype: object

<div class='alert alert-block alert-info'> 
The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.
</div>

In [4]:
df = pd.DataFrame(df, columns=['Text'])

In [5]:
df.loc[0:100]

,Text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Services: None\n
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
5,.Per 7/06/79 Movement D/O note:\n
6,"4, 5/18/78 Patient's thoughts about current substance abuse:\n"
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnosis Interview\n
8,3/7/86 SOS-10 Total Score:\n
9,(4/10/71)Score-1Audit C Score Current:\n


In [6]:
df_copy = df.copy()

## List of variations

<div class='alert alert-block alert-info'>
<li> 04/20/2009; 04/20/09; 4/20/09; 4/3/09</li>
<li> Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;</li>
<li> 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009</li>
<li> Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009</li>
<li> Feb 2009; Sep 2009; Oct 2010</li>
<li> 6/2008; 12/2009</li>
<li> 2009; 2010</li>
</div>    


<div class='alert alert-block alert-danger'>
 <h3> Helper Functions</h3>
</div>

In [7]:
def removeIdentified(df_original, df_toremove):
    #Each index in created df is a tuple, the first element of this tuple is the index of the original table
    captured_index  = pd.Series([i[0] for i in df_toremove.index])
    print("Number removed: ", captured_index.shape)
    df_original = df_original[~df_original.index.isin(captured_index)]
    print("Number left : " ,df_original.shape)#(379, 2)
    return df_original


def adjustDayMonthYear(df_input):
    if 'Day' not in df_input.columns :
        df_input['Day'] = '01'
    if 'Month' not in df_input.columns:
        df_input['Month'] = '01'
    df_input['Year'].apply(lambda x: "19"+ x if len(x) == 2 else x)
    return df_input
    
def createFullDate(df_input):
    #Strip Month , year and date off , . -
    transTable = str.maketrans('', '', r',./')
    df_input['Day'] = df_input['Day'].str.translate(transTable)
    df_input['Month'] = df_input['Month'].str.translate(transTable)
    df_input['Year'] = df_input['Year'].str.translate(transTable)
    df_input['FullDate'] = df_input['Year'] +'-' + df_input['Month']+ '-'+df_input['Day']
    df_input['FullDate'] = pd.to_datetime(df_input['FullDate'])
    df_input = df_input[['FullDate', 'Day', 'Month', 'Year']]
    return df_input

def getDFdetails(df_input):
    print(df_input.name)
    print(df_input.shape)
    print(df_input.head())
      
    

<div class='alert alert-block alert-success'>
    <h3> Procedure 1 </h3>
    Check for dates with all three components
    <li> 04/20/2009; 04/20/09; 4/20/09; 4/3/09, 1-14-81 </li>
</div>

In [8]:
df_extract_numerical = df_copy['Text'].str.extractall(r'(?P<FullDate>(?P<Month>\d?\d)[/-](?P<Day>\d?\d)[/-](?P<Year>\d?\d?\d\d))')

#check for year equal to three digits
#df_numerical[(df_numerical['Year'].str.len() == 3)]
#72	1	9/36/308	9	36	

# This is the CBC count and so drop these
#df_copy[df_copy['Text'].str.contains('9/36/308')]

#Remove  if dates year is NOT 2 digits or is not 4 digits
df_extract_numerical = df_extract_numerical[(df_extract_numerical['Year'].str.len() == 2) | (df_extract_numerical['Year'].str.len() == 4)]
df_extract_numerical.name = 'df_extract_numerical'
#df_copy.iloc[106]

In [9]:
getDFdetails(df_extract_numerical)

df_extract_numerical
(125, 4)
         FullDate Month Day Year
  match                         
0 0      03/25/93    03  25   93
1 0       6/18/85     6  18   85
2 0        7/8/71     7   8   71
3 0       9/27/75     9  27   75
4 0        2/6/96     2   6   96


<div class='alert alert-block alert-warning'>
 <li> Reduce the size of the original dataframe by the identified rows  </li> 
</div>    

In [10]:
df_copy = removeIdentified(df_copy,df_extract_numerical)

Number removed:  (125,)
Number left :  (375, 1)


<div class='alert alert-block alert-success'>
        <h3> Procedure 2 </h3>
        <li> Looking for months spelled out  </li>  
<li> Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; </li>
</div>    

In [11]:
from calendar import month_name

patternlist =  ["(" + a[0:3]+"(" + a[3:] + "|.)?,?)" if len(a) > 3 else a for a in month_name ]
patternlist = patternlist[1:]
totalpattern = "|".join(patternlist) 
#print(patternlist)
print(totalpattern)



(Jan(uary|.)?,?)|(Feb(ruary|.)?,?)|(Mar(ch|.)?,?)|(Apr(il|.)?,?)|May|(Jun(e|.)?,?)|(Jul(y|.)?,?)|(Aug(ust|.)?,?)|(Sep(tember|.)?,?)|(Oct(ober|.)?,?)|(Nov(ember|.)?,?)|(Dec(ember|.)?,?)


In [12]:
df_extract_monthname = df_copy['Text'].str.extractall(r'(?P<FullDate>(?P<Month>(Jan(?:uary|.)?,?)|(Feb(?:ruary|.)?,?)|(Mar(?:ch|.)?,?)|(Apr(?:il|.)?,?)|May|(Jun(?:e|.)?,?)|(Jul(?:y|.)?,?)|(Aug(?:ust|.)?,?)|(Sep(?:tember|.)?,?)|(Oct(?:ober|.)?,?)|(Nov(?:ember|.)?,?)|(Dec(?:ember|.)?,?))[\s-](?P<Day>\d?\d),?[\s-](?P<Year>\d\d\d\d))')
df_extract_monthname.name = 'df_extract_monthname'

In [13]:
getDFdetails(df_extract_monthname)

df_extract_monthname
(32, 15)
                    FullDate     Month    2         3    4      5    6    7  \
    match                                                                     
194 0         April 11, 1990     April  NaN       NaN  NaN  April  NaN  NaN   
195 0           May 30, 2001       May  NaN       NaN  NaN    NaN  NaN  NaN   
196 0           Feb 18, 1994       Feb  NaN       Feb  NaN    NaN  NaN  NaN   
197 0      February 18, 1981  February  NaN  February  NaN    NaN  NaN  NaN   
199 0            Jan 24 1986       Jan  Jan       NaN  NaN    NaN  NaN  NaN   

             8    9   10   11   12 Day  Year  
    match                                     
194 0      NaN  NaN  NaN  NaN  NaN  11  1990  
195 0      NaN  NaN  NaN  NaN  NaN  30  2001  
196 0      NaN  NaN  NaN  NaN  NaN  18  1994  
197 0      NaN  NaN  NaN  NaN  NaN  18  1981  
199 0      NaN  NaN  NaN  NaN  NaN  24  1986  


<div class='alert alert-block alert-warning'>
<li>Reduce the size of the original dataframe by the identified rows  </li>  
</div>  

In [14]:
df_copy = removeIdentified(df_copy,df_extract_monthname)

Number removed:  (32,)
Number left :  (343, 1)


<div class='alert alert-block alert-success'>
    <h3> Procedure 3 </h3>
    <li> Retrieve these : 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009</li>
</div>

In [15]:
df_extract_monthname_middle = df_copy['Text'].str.extractall(r'(?P<FullDate>(?P<Day>\d?\d)\s(?P<Month>(Jan(?:uary|.)?,?)|(Feb(?:ruary|.)?,?)|(Mar(?:ch|.)?,?)|(Apr(?:il|.)?,?)|May|(Jun(?:e|.)?,?)|(Jul(?:y|.)?,?)|(Aug(?:ust|.)?,?)|(Sep(?:tember|.)?,?)|(Oct(?:ober|.)?,?)|(Nov(?:ember|.)?,?)|(Dec(?:ember|.)?,?))[\s-](?P<Year>\d\d\d\d))')
df_extract_monthname_middle.name = 'df_extract_monthname_middle'

In [16]:
getDFdetails(df_extract_monthname_middle)

df_extract_monthname_middle
(69, 15)
               FullDate Day Month    3    4    5    6     7    8    9   10  \
    match                                                                    
125 0       24 Jan 2001  24   Jan  Jan  NaN  NaN  NaN   NaN  NaN  NaN  NaN   
126 0       10 Sep 2004  10   Sep  NaN  NaN  NaN  NaN   NaN  NaN  NaN  Sep   
127 0       26 May 1982  26   May  NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   
128 0      28 June 2002  28  June  NaN  NaN  NaN  NaN  June  NaN  NaN  NaN   
129 0       06 May 1972  06   May  NaN  NaN  NaN  NaN   NaN  NaN  NaN  NaN   

            11   12   13  Year  
    match                       
125 0      NaN  NaN  NaN  2001  
126 0      NaN  NaN  NaN  2004  
127 0      NaN  NaN  NaN  1982  
128 0      NaN  NaN  NaN  2002  
129 0      NaN  NaN  NaN  1972  


<div class='alert alert-block alert-warning'>
   Reduce the size of the original dataframe by the identified rows
</div>

In [17]:
df_copy = removeIdentified(df_copy,df_extract_monthname_middle)

Number removed:  (69,)
Number left :  (274, 1)


<div class='alert alert-block alert-success'>
Check for dates with only two of the three components
    <h3> Procedure 4 </h3>
    <li>Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009</li>
<li> Does not start with one or two digits, but has month with one or two digits and years with four digits </li>    
</div>

In [ ]:
# print(df_copy[df_copy['Text'].str.contains('\d?\dth', regex=True)])
# print(df_copy[df_copy['Text'].str.contains('\d?\dst', regex=True)])
# print(df_copy[df_copy['Text'].str.contains('\d?\dth', regex=True)])

# pd.set_option('max_colwidth', 1000)

In [18]:
df_extract_datequalifier = df_copy['Text'].str.extractall(r'(?P<FullDate>(?P<Month>(Jan(?:uary|.)?,?)|(Feb(?:ruary|.)?,?)|(Mar(?:ch|.)?,?)|(Apr(?:il|.)?,?)|May|(Jun(?:e|.)?,?)|(Jul(?:y|.)?,?)|(Aug(?:ust|.)?,?)|(Sep(?:tember|.)?,?)|(Oct(?:ober|.)?,?)|(Nov(?:ember|.)?,?)|(Dec(?:ember|.)?,?))\s(?P<Day>\d?\d(?:nd|st|th|rd),)\s(?P<Year>\d\d\d\d))')
df_extract_datequalifier.name = 'df_extract_datequalifier'

In [19]:
getDFdetails(df_extract_datequalifier)

df_extract_datequalifier
(0, 15)
Empty DataFrame
Columns: [FullDate, Month, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, Day, Year]
Index: []


<div class='alert alert-block alert-warning'>
   Reduce the size of the original dataframe by the identified rows
</div>

In [ ]:
#df_copy = removeIdentified(df_copy,df_datequalifier)

<div class='alert alert-block alert-success'>

<h3>Procedure 5</h3>
<li>Check for dates with only two of the three components</li>
<li>Feb 2009; Sep 2009; Oct 2010</li>
<li>Does not start with one or two digits, but has month with one or two digits and years with four digits</li>
</div>

In [20]:
df_copy.shape

(274, 1)

In [21]:
df_extract_monthyear = df_copy['Text'].str.extractall(r'(?P<FullDate>(?P<Month>Jan\s|Feb\s|Mar\s|Apr\s|May\s|Jun\s|Jul\s|Aug\s|Sep\s|Oct\s|Nov\s|Dec\s)(?P<Year>\d\d\d\d))')
df_extract_monthyear.name = 'df_extract_monthyear'

In [22]:
getDFdetails(df_extract_monthyear)

df_extract_monthyear
(47, 3)
           FullDate Month  Year
    match                      
230 0      May 1986  May   1986
231 0      May 2016  May   2016
234 0      Jul 2003  Jul   2003
235 0      Oct 2015  Oct   2015
236 0      May 1995  May   1995


<div class='alert alert-block alert-warning'>
   Reduce the size of the original dataframe by the identified rows
</div>

In [23]:
df_copy = removeIdentified(df_copy,df_extract_monthyear)

Number removed:  (47,)
Number left :  (227, 1)


<div class='alert alert-block alert-success'>
<h3>Procedure 6 </h3>
<li>Check for dates with only two of the three components</li>
<li>    6/2008; 12/2009</li>
<li>Does not start with one or two digits, but has month with one or two digits and years with four digits</li>
</div>


In [24]:
df_extract_numerical_month_year = df_copy['Text'].str.extractall(r'(?P<FullDate>(?P<Month>\b\d?\d)\/(?P<Year>\d\d\d\d)\b)')
df_extract_numerical_month_year.name = 'df_extract_numerical_month_year'

In [25]:
getDFdetails(df_extract_numerical_month_year)

df_extract_numerical_month_year
(110, 3)
          FullDate Month  Year
    match                     
343 0       6/1998     6  1998
344 0       6/2005     6  2005
345 0      10/1973    10  1973
346 0       9/2005     9  2005
347 0      03/1980    03  1980


<div class='alert alert-block alert-warning'>
   Reduce the size of the original dataframe by the identified rows
</div>

In [26]:
df_copy = removeIdentified(df_copy,df_extract_numerical_month_year)

Number removed:  (110,)
Number left :  (117, 1)



<div class='alert alert-block alert-success'>
    <h3> Procedure 7 </h3>
    <li>Check for dates with only year</li>
<li>2008; 2009 </li>
<li> Word boundary followed by year </li>    
</div>

In [27]:
df_extract_just_year = df_copy['Text'].str.extractall(r'(?P<Year>\d{4,4})[^\d]?')
df_extract_just_year = df_extract_just_year[df_extract_just_year['Year'] < '3000']
df_extract_just_year.name = 'df_extract_just_year'
getDFdetails(df_extract_just_year)

df_extract_just_year
(117, 1)
           Year
    match      
198 0      2013
227 0      2011
228 0      1985
229 0      2011
232 0      1977


<div class='alert alert-block alert-warning'>
   Reduce the size of the original dataframe by the identified rows
</div>

In [28]:
df_copy = removeIdentified(df_copy,df_extract_just_year)

Number removed:  (117,)
Number left :  (0, 1)


In [29]:
listDF = %who_ls DataFrame
#Get the list of dataframe create and clean them up
names = [dfName for dfName in listDF if "extract" in dfName]
print(names)

# for name in names:
#      return_df = adjustDayMonthYear(df_extract_dict[name])
#      return_df = createFullDate(return_df)
#      df_extract_dict[name] = return_df   
#      df_extract_dict[name].reset_index(inplace=True) 
#      df_extract_dict[name].rename(columns={'level_0':'OriginalIndex'}, inplace=True)
        
# for name in names:
#     adjustDayMonthYear(df_extract_dict[name])
#     createFullDate(df_extract_dict[name])
    
df_extract_just_year = adjustDayMonthYear(df_extract_just_year) 
df_extract_monthname= adjustDayMonthYear(df_extract_monthname)    
df_extract_monthname_middle = adjustDayMonthYear(df_extract_monthname_middle)    
df_extract_monthyear = adjustDayMonthYear(df_extract_monthyear)    
df_extract_numerical= adjustDayMonthYear(df_extract_numerical)    
df_extract_numerical_month_year = adjustDayMonthYear(df_extract_numerical_month_year)    
   
df_extract_just_year = createFullDate(df_extract_just_year) 
df_extract_monthname = createFullDate(df_extract_monthname)    
df_extract_monthname_middle = createFullDate(df_extract_monthname_middle)    
df_extract_monthyear = createFullDate(df_extract_monthyear)    
df_extract_numerical = createFullDate(df_extract_numerical)    
df_extract_numerical_month_year = createFullDate(df_extract_numerical_month_year)    

    


['df_extract_datequalifier', 'df_extract_just_year', 'df_extract_monthname', 'df_extract_monthname_middle', 'df_extract_monthyear', 'df_extract_numerical', 'df_extract_numerical_month_year']


In [35]:
new_df = pd.concat([df_extract_just_year, df_extract_monthname, df_extract_monthname_middle, df_extract_monthyear, df_extract_numerical, df_extract_numerical_month_year])

                



In [36]:
new_df

,,FullDate,Day,Month,Year
,match,,,,
198,0,2013-01-01,01,01,2013
227,0,2011-01-01,01,01,2011
228,0,1985-01-01,01,01,1985
229,0,2011-01-01,01,01,2011
232,0,1977-01-01,01,01,1977
233,0,1990-01-01,01,01,1990
237,0,1976-01-01,01,01,1976
238,0,1995-01-01,01,01,1995
240,0,2011-01-01,01,01,2011


In [ ]:
def date_sorter():
    
    # Your code here
    
    return # Your answer here









# End of assignment

#  Exercises from the Course

In [ ]:
import pandas as pd

time_sentences = ["Monday: The doctor's appointment is at 2:45pm.", 
                  "Tuesday: The dentist's appointment is at 11:30 am.",
                  "Wednesday: At 7:00pm, there is a basketball game!",
                  "Thursday: Be back home by 11:15 pm at the latest.",
                  "Friday: Take the train at 08:10 am, arrive at 09:00am."]

df = pd.DataFrame(time_sentences, columns=['text'])
df

####  Find the number of characters for each string in df['text']

In [ ]:

df = df.assign(text_length = df['text'].apply(len))

#### Find the number of tokens for each string in df['text']

In [ ]:

# df.assign(split_text = df['text'].str.split())
df_expanded = df['text'].str.split(expand=True)

df_expanded


####  Find which entries contain the word 'appointment'

In [ ]:
df = df.assign(text_contains_appointment = df['text'].str.contains('appointment'))

####  Find how many times a digit occurs in each string

<div class="alert alert-block alert-success">
<h5>Two methods shown - One gets the count of digits and the other uses a powerful function "enumerate" to loop through the string . It allows us to loop over something and have an automatic counter.</h5>
</div>

In [ ]:
df = df.assign(contains_digits = df['text'].str.count(r'\d'))

In [ ]:
def indexOfDigit(text):
    return ({c: i  for i, c in enumerate(text) if c.isdigit()})
    
df['text'].apply(indexOfDigit)

####  Find all occurances of the digits

In [ ]:

df['text'].str.findall(r'\d')

In [ ]:
Image('FIG3.png', width=400, height=400)

In [ ]:
# group and find the hours and minutes
df['text'].str.findall(r'(\d?\d):(\d?\d)')

In [ ]:
# replace weekdays with '???'
df['text'].str.replace(r'\w+day\b', '???')

In [ ]:
# replace weekdays with 3 letter abbrevations
df['text'].str.replace(r'(\w+day\b)', lambda x: x.groups()[0][:3])

In [ ]:
# create new columns from first match of extracted groups
df['text'].str.extract(r'(\d?\d):(\d\d)')

In [ ]:
# extract the entire time, the hours, the minutes, and the period
df['text'].str.extractall(r'((\d?\d):(\d\d) ?([ap]m))')

In [ ]:
# extract the entire time, the hours, the minutes, and the period with group names
df['text'].str.extractall(r'(?P<time>(?P<hour>\d?\d):(?P<minute>\d\d) ?(?P<period>[ap]m))')

Adding a new cell